In [ ]:
import os
import pinecone
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env')

# get api key from app.pinecone.io for .env file
lines = open('../.env').read().splitlines()
for line in lines:
    if line.startswith('PINECONE_API_KEY'):
        api_key = line.split('=')[1]
        break
api_key = api_key or os.environ.get('PINECONE_API_KEY')
# find your environment next to the api key in pinecone console
env = os.environ.get('PINECONE_ENVIRONMENT') or 'gcp-starter'

pinecone.init(
    api_key=api_key,
    environment=env
)

In [ ]:
index_name = 'test-1'
pinecone.list_indexes()

['test-1']

In [ ]:
# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=1,
        metric='cosine'
    )

# now connect to the index
index = pinecone.Index(index_name)

In [ ]:
# create list of random 1 d vectors to upsert, random metadata from keys a,b,c,d set to X or Y randomly
# random numbers are between 0 and 1
import numpy as np

vectors = []

vectors = [
    {
        "id": str(i),
        "values": np.np.random.rand(1),
        "metadata": {
            "a": "X" if np.random.rand(1) > 0.5 else "Y",
            "b": "X" if np.random.rand(1) > 0.5 else "Y",
            "c": "X" if np.random.rand(1) > 0.5 else "Y",
            "d": "X" if np.random.rand(1) > 0.5 else "Y",
        },
    }
    for i in range(1000,1000_000)
]



In [ ]:
from tqdm import tqdm
batch_size = 1000
for i in tqdm(range(0, len(vectors), batch_size)):
    batch_vectors = vectors[i:i+batch_size]
    index.upsert(batch_vectors)

# upsert vectors
# index.upsert(vectors)

 12%|█▏        | 115/999 [01:23<10:45,  1.37it/s]


ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'Content-Length': '135', 'x-pinecone-request-latency-ms': '102', 'date': 'Wed, 20 Dec 2023 09:35:08 GMT', 'x-envoy-upstream-service-time': '23', 'server': 'envoy', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"code":9,"message":"Starter index record limit reached. Current number: 104000, records in request: 1000, limit: 100000","details":[]}
